In [ ]:
# import mediapipe as mp
import cv2
import numpy as np
import uuid
import os
import pandas as pd
import copy
import time

from numpy import *
from pandas import *
from google.colab import files, drive
from os import listdir

# 데이터 파일 불러옴

In [ ]:
df = pd.read_excel('데이터 파일 이름')

In [ ]:
upload = files.upload()

Saving data_all.xlsx to data_all (1).xlsx


In [ ]:
df = pd.read_excel('data_all.xlsx')
df

,x_0,y_0,z_0,x_1,y_1,z_1,x_2,y_2,z_2,x_3,y_3,z_3,x_4,y_4,z_4,x_5,y_5,z_5,x_6,y_6,z_6,x_7,y_7,z_7,x_8,y_8,z_8,x_9,y_9,z_9,x_10,y_10,z_10,x_11,y_11,z_11,x_12,y_12,z_12,x_13,y_13,z_13,x_14,y_14,z_14,x_15,y_15,z_15,x_16,y_16,z_16,d_ratio,d_ratio_sq,sin dif,엄지 중지 내적,검지 중지 내적,y,image
0,0.802822,0.683172,-0.000059,0.796290,0.661867,-0.146656,0.757204,0.551149,-0.192669,0.706601,0.448169,-0.197359,0.655755,0.354045,-0.204960,0.790932,0.368932,-0.129292,0.698669,0.313862,-0.172512,0.632303,0.337803,-0.203215,0.582439,0.375781,-0.222803,0.767720,0.376558,-0.066249,0.674490,0.323270,-0.119961,0.611638,0.368772,-0.157459,0.563561,0.424743,-0.177124,0.747185,0.405188,-0.011417,0.671273,0.403154,-0.081756,0.651415,0.484210,-0.115277,0.636442,0.549586,-0.129835,0,0,0,0,0,1,correct18.MP4_000003.040.jpg
1,0.802112,0.672191,-0.000017,0.795411,0.642390,-0.143206,0.759011,0.525775,-0.185622,0.707753,0.423363,-0.191975,0.659722,0.332323,-0.206871,0.805196,0.353425,-0.120753,0.716508,0.304782,-0.167444,0.651719,0.327338,-0.198758,0.602915,0.368467,-0.217108,0.774002,0.358659,-0.055587,0.680201,0.312513,-0.119463,0.617398,0.358546,-0.161553,0.567457,0.416868,-0.181146,0.745328,0.382709,-0.000876,0.664769,0.395086,-0.077951,0.645512,0.484567,-0.118999,0.630237,0.551435,-0.134810,0,0,0,0,0,1,correct18.MP4_000003.160.jpg
2,0.834878,0.630622,-0.000036,0.832766,0.562037,-0.129846,0.795560,0.451895,-0.171299,0.735935,0.353962,-0.183522,0.687844,0.256539,-0.208985,0.823451,0.278121,-0.086189,0.737856,0.238672,-0.133510,0.679090,0.265810,-0.178718,0.640781,0.307479,-0.210603,0.787495,0.296410,-0.029600,0.700509,0.257458,-0.090715,0.640829,0.308430,-0.147433,0.605954,0.356631,-0.180794,0.757250,0.327631,0.016060,0.680012,0.327506,-0.052788,0.646495,0.405078,-0.102559,0.631131,0.453318,-0.129861,0,0,0,0,0,1,correct18.MP4_000003.640.jpg
3,0.840423,0.611011,-0.000028,0.835926,0.539953,-0.117908,0.799676,0.425365,-0.148123,0.748405,0.330711,-0.154980,0.702745,0.246166,-0.177059,0.816756,0.283106,-0.073382,0.738299,0.233189,-0.122390,0.679344,0.252439,-0.170702,0.635466,0.291793,-0.204810,0.782272,0.301644,-0.018568,0.701290,0.262287,-0.068624,0.643236,0.303859,-0.120193,0.605661,0.355507,-0.151100,0.752858,0.330315,0.024647,0.682568,0.320169,-0.040536,0.645990,0.383178,-0.094835,0.622334,0.434460,-0.124643,0,0,0,0,0,1,correct18.MP4_000003.761.jpg
4,0.846491,0.607810,-0.000035,0.832838,0.530135,-0.118343,0.795919,0.423277,-0.151772,0.743653,0.332124,-0.160703,0.700847,0.235392,-0.184964,0.826159,0.247272,-0.073614,0.737623,0.216855,-0.118516,0.674505,0.248014,-0.160364,0.632795,0.280961,-0.187706,0.794500,0.274452,-0.015906,0.707748,0.258341,-0.073416,0.660718,0.329992,-0.122204,0.640665,0.384702,-0.140170,0.766458,0.309036,0.030765,0.697225,0.319558,-0.036545,0.681332,0.406327,-0.077488,0.680488,0.460202,-0.090807,0,0,0,0,0,1,correct18.MP4_000003.880.jpg
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2080,0.683287,0.925784,-0.000051,0.683226,0.808627,-0.066687,0.691961,0.666358,-0.079175,0.706048,0.554577,-0.080995,0.723846,0.447428,-0.080970,0.687060,0.600681,0.005907,0.703182,0.481403,0.028084,0.724452,0.415654,0.039631,0.746156,0.370086,0.050197,0.694650,0.627415,0.051697,0.701560,0.514276,0.090203,0.717508,0.461997,0.105240,0.733055,0.419804,0.115534,0.701844,0.655029,0.089303,0.708349,0.560268,0.130411,0.719038,0.512440,0.142770,0.729677,0.475407,0.150917,0,0,0,0,0,0,not_correct7.mp4_000004.574.jpg
2081,0.689466,0.911463,-0.000065,0.662194,0.809515,-0.053108,0.658066,0.686111,-0.058303,0.662225,0.588759,-0.051490,0.672364,0.500283,-0.043548,0.684964,0.597477,0.009075,0.699973,0.486054,0.027304,0.711765,0.429146,0.027077,0.720656,0.390802,0.025083,0.695864,0.605227,0.048374,0.710104,0.490839,0.079638,0.720213,0.433627,0.090777,0.731818,0.400537,0.096398,0.703909,0.623557,0

In [ ]:
for i in range(17):
  df = df.drop(['z_{}'.format(i)], axis=1)

In [ ]:
df = df.drop(['d_ratio',	'd_ratio_sq',	'sin dif',	'엄지 중지 내적',	'검지 중지 내적',	'y'], axis=1)

In [ ]:
df

,x_0,y_0,x_1,y_1,x_2,y_2,x_3,y_3,x_4,y_4,x_5,y_5,x_6,y_6,x_7,y_7,x_8,y_8,x_9,y_9,x_10,y_10,x_11,y_11,x_12,y_12,x_13,y_13,x_14,y_14,x_15,y_15,x_16,y_16,image
0,0.802822,0.683172,0.796290,0.661867,0.757204,0.551149,0.706601,0.448169,0.655755,0.354045,0.790932,0.368932,0.698669,0.313862,0.632303,0.337803,0.582439,0.375781,0.767720,0.376558,0.674490,0.323270,0.611638,0.368772,0.563561,0.424743,0.747185,0.405188,0.671273,0.403154,0.651415,0.484210,0.636442,0.549586,correct18.MP4_000003.040.jpg
1,0.802112,0.672191,0.795411,0.642390,0.759011,0.525775,0.707753,0.423363,0.659722,0.332323,0.805196,0.353425,0.716508,0.304782,0.651719,0.327338,0.602915,0.368467,0.774002,0.358659,0.680201,0.312513,0.617398,0.358546,0.567457,0.416868,0.745328,0.382709,0.664769,0.395086,0.645512,0.484567,0.630237,0.551435,correct18.MP4_000003.160.jpg
2,0.834878,0.630622,0.832766,0.562037,0.795560,0.451895,0.735935,0.353962,0.687844,0.256539,0.823451,0.278121,0.737856,0.238672,0.679090,0.265810,0.640781,0.307479,0.787495,0.296410,0.700509,0.257458,0.640829,0.308430,0.605954,0.356631,0.757250,0.327631,0.680012,0.327506,0.646495,0.405078,0.631131,0.453318,correct18.MP4_000003.640.jpg
3,0.840423,0.611011,0.835926,0.539953,0.799676,0.425365,0.748405,0.330711,0.702745,0.246166,0.816756,0.283106,0.738299,0.233189,0.679344,0.252439,0.635466,0.291793,0.782272,0.301644,0.701290,0.262287,0.643236,0.303859,0.605661,0.355507,0.752858,0.330315,0.682568,0.320169,0.645990,0.383178,0.622334,0.434460,correct18.MP4_000003.761.jpg
4,0.846491,0.607810,0.832838,0.530135,0.795919,0.423277,0.743653,0.332124,0.700847,0.235392,0.826159,0.247272,0.737623,0.216855,0.674505,0.248014,0.632795,0.280961,0.794500,0.274452,0.707748,0.258341,0.660718,0.329992,0.640665,0.384702,0.766458,0.309036,0.697225,0.319558,0.681332,0.406327,0.680488,0.460202,correct18.MP4_000003.880.jpg
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2080,0.683287,0.925784,0.683226,0.808627,0.691961,0.666358,0.706048,0.554577,0.723846,0.447428,0.687060,0.600681,0.703182,0.481403,0.724452,0.415654,0.746156,0.370086,0.694650,0.627415,0.701560,0.514276,0.717508,0.461997,0.733055,0.419804,0.701844,0.655029,0.708349,0.560268,0.719038,0.512440,0.729677,0.475407,not_correct7.mp4_000004.574.jpg
2081,0.689466,0.911463,0.662194,0.809515,0.658066,0.686111,0.662225,0.588759,0.672364,0.500283,0.684964,0.597477,0.699973,0.486054,0.711765,0.429146,0.720656,0.390802,0.695864,0.605227,0.710104,0.490839,0.720213,0.433627,0.731818,0.400537,0.703909,0.623557,0.715664,0.525182,0.722518,0.468309,0.728340,0.438733,not_correct7.mp4_000005.110.jpg
2082,0.748213,0.857295,0.711621,0.746825,0.703696,0.626139,0.706495,0.528605,0.715309,0.446488,0.718847,0.552104,0.683949,0.455952,0.641702,0.441638,0.609599,0.460535,0.722917,0.556032,0.679210,0.440653,0.624662,0.430447,0.581561,0.463786,0.727055,0.570289,0.682644,0.477938,0.628725,0.473683,0.588093,0.493850,not_correct7.mp4_000005.377.jpg
2083,0.725273,0.837049,0.669997,0.710730,0.658297,0.590605,0.659387,0.499487,0.683194,0.432289,0.674882,0.507352,0.646205,0.392054,0.607197,0.391531,0.580782,0.435100,0.696248,0.510634,0.670623,0.388990,0.624398,0.417362,0.590639,0.475444,0.717677,0.520330,0.697354,0.409094,0.654995,0.441637,0.621378,0.496652,not_correct7.mp4_000005.507.jpg


이 상태가 어플에서 새로운 인풋이 들어왔을때 만든 엑셀 파일

# 불러온 데이터 파일에 preprocessing 할 column 추가 및 정렬

In [ ]:
def add_columns():
  global df
  df["d ratio"] = 0
  df["d ratio sq"] = 0
  df["sin dif"] = 0
  df["엄지 중지 내적"] = 0
  df["검지 중지 내적"] = 0
  df = df.sort_values(by=["image"], ascending=[True])
  # return df
  pass

# 행 추가 함수

# 전처리 함수들

In [ ]:
# d ratio, d ratio sq 만들어 주는 함수

def d_ratio():
  global df

  # df.iloc[i, 34] = ((df.iloc[i, 24]-df.iloc[i, 32])**2+(df.iloc[i, 25]-df.iloc[i, 33])**2)/((df.iloc[i, 24]-df.iloc[i, 16])**2+(df.iloc[i, 25]-df.iloc[i, 17])**2)
  df['d ratio'] = ((df['x_12']-df['x_16'])**2 + (df['y_12']-df['y_16'])**2)**0.5 / ((df['x_8']-df['x_12'])**2 + (df['y_8']-df['y_12'])**2)**0.5   # x,y 만 반영한 d ratio
  df['d ratio sq'] = ((df['x_12']-df['x_16'])**2 + (df['y_12']-df['y_16'])**2) / ((df['x_8']-df['x_12'])**2 + (df['y_8']-df['y_12'])**2)   # x,y 만 반영한 d ratio 의 제곱

  # return df
  pass

In [ ]:
# 벡터화 시키는 함수

def vectorization():
  global df, df_vector
  df_vector = df.copy()

  column = []
  for i in range (0, 16):
    column.append('x_{1}-{0}'.format(i,i+1))
    column.append('y_{1}-{0}'.format(i,i+1))
  column.append('image')
  column.append('d_ratio')
  column.append('d_ratio_sq')
  column.append('sin dif')
  column.append('엄지 중지 내적')
  column.append('검지 중지 내적')
  
 
  for i in range(0,16):
    df_vector['x_{0}'.format(i)] = df['x_{0}'.format(i+1)]-df['x_{0}'.format(i)]
    df_vector['y_{0}'.format(i)] = df['y_{0}'.format(i+1)]-df['y_{0}'.format(i)]

  df_vector = df_vector.drop(['x_16', 'y_16'], axis=1)

  df_vector.columns = column

  df_vector = pd.DataFrame(df_vector)
  # return df_vector
  pass

In [ ]:
# vector size 구하는 함수

def vectorsize():
  global df_size
  df_size = df_vector.copy()

  column_size =[]
  for i in range(0,16):
    column_size.append('size_{0}-{1}'.format(i+1,i))

  for i in range(0,16):
    df_size['x_{0}-{1}'.format(i+1, i)] = (df_vector['x_{0}-{1}'.format(i+1, i)]**2+df_vector['y_{0}-{1}'.format(i+1, i)]**2)**0.5
    df_size = df_size.drop(['y_{0}-{1}'.format(i+1, i)], axis=1)
  
  df_size = df_size.drop(['image',	'd_ratio',	'd_ratio_sq',	'sin dif','엄지 중지 내적',	'검지 중지 내적'], axis=1)
  df_size.columns = column_size
  pass

In [ ]:
# vector size를 통해 단위벡터화 시키는 함수

def unitvector():
  global df_unitvector
  df_unitvector = df_vector.copy()

  for i in range(0,16):
    df_unitvector['x_{0}-{1}'.format(i+1, i)] = df_vector['x_{0}-{1}'.format(i+1, i)]/df_size['size_{0}-{1}'.format(i+1, i)]
    df_unitvector['y_{0}-{1}'.format(i+1, i)] = df_vector['y_{0}-{1}'.format(i+1, i)]/df_size['size_{0}-{1}'.format(i+1, i)]
    
  pass

In [ ]:
# 중지 약지의 sin 값 차이 데이터 만드는 함수

def sin_dif():
  global df_vector
  df_vector['cos 7-6, 11-10'] =0
  df_vector['sin 7-6, 11-10'] = 0
  df_vector['cos 11-10, 15-14'] = 0
  df_vector['sin 11-10, 15-14'] = 0

  df_vector['cos 7-6, 11-10'] = df_unitvector['x_7-6']*df_unitvector['x_11-10'] + df_unitvector['y_7-6']*df_unitvector['y_11-10']
  df_vector['sin 7-6, 11-10'] = df_unitvector['y_7-6']*df_unitvector['x_11-10'] - df_unitvector['x_7-6']*df_unitvector['y_11-10']
  df_vector['cos 11-10, 15-14'] = df_unitvector['x_11-10']*df_unitvector['x_15-14'] + df_unitvector['y_11-10']*df_unitvector['y_15-14']
  df_vector['sin 11-10, 15-14'] = df_unitvector['y_11-10']*df_unitvector['x_15-14'] - df_unitvector['x_11-10']*df_unitvector['y_15-14']

  df_vector['sin dif'] = df_vector['sin 7-6, 11-10']*df_vector['cos 11-10, 15-14'] - df_vector['cos 7-6, 11-10']*df_vector['sin 11-10, 15-14']

  df_vector = df_vector.drop(['cos 7-6, 11-10',	'sin 7-6, 11-10',	'cos 11-10, 15-14',	'sin 11-10, 15-14'], axis=1)

  pass

In [ ]:
# 엄지 중지 첫 마디 내적값 만드는 함수

def dotproduct_tm():
  global df_unitvector, df_vector
  
  df_vector['엄지 중지 내적'] = df_unitvector['x_4-3']*df_unitvector['x_12-11'] + df_unitvector['y_4-3']*df_unitvector['y_12-11']

  pass

In [ ]:
# 검지 중지 첫 마디 내적값 만드는 함수

def dotproduct_im():
  global df_unitvector, df_vector

  df_vector['검지 중지 내적'] = df_unitvector['x_8-7']*df_unitvector['x_12-11'] + df_unitvector['y_8-7']*df_unitvector['y_12-11']

  pass

In [ ]:
# 16-15 크기 정규화 시키는 함수

def normed_1615():
  global df_vector, df_size

  for n in range(len(df)):
    for i in range(16):
      df_vector.iloc[n,2*i] = df_vector.iloc[n,2*i]/df_size.iloc[n, 15]
      df_vector.iloc[n,2*i+1] = df_vector.iloc[n,2*i+1]/df_size.iloc[n, 15]

  pass

In [ ]:
# 3-2 벡터를 기준으로 회전 정규화 시키는 함수

def rotate_32():
  global df_vector, df_unitvector, df_vector_final
  df_vector_final = df_vector.copy()

  for i in range(0,16):
    df_vector_final['x_{0}-{1}'.format(i+1, i)] = df_vector['x_{0}-{1}'.format(i+1, i)]*df_unitvector['x_3-2'] + df_vector['y_{0}-{1}'.format(i+1, i)]*df_unitvector['y_3-2']
    df_vector_final['y_{0}-{1}'.format(i+1, i)] = -df_vector['x_{0}-{1}'.format(i+1, i)]*df_unitvector['y_3-2'] + df_vector['y_{0}-{1}'.format(i+1, i)]*df_unitvector['x_3-2']

  pass

In [ ]:
# df_vector_final = df_vector_final.sort_values(by=["image"], ascending=[True])

# df_save = pd.DataFrame(df_vector_final)

# with pd.ExcelWriter('preprocessed_0826_all.xlsx') as writer:
#   df_save.to_excel(writer, sheet_name='data',index=False, header=True)
  
# files.download('preprocessed_0826_all.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
def preprocess():
  global df

  add_columns()
  d_ratio()
  vectorization()
  vectorsize()
  unitvector()
  sin_dif()
  dotproduct_tm()
  dotproduct_im()
  normed_1615()
  # rotate_32()

  return df_vector_final

In [ ]:
preprocess()

,x_1-0,y_1-0,x_2-1,y_2-1,x_3-2,y_3-2,x_4-3,y_4-3,x_5-4,y_5-4,x_6-5,y_6-5,x_7-6,y_7-6,x_8-7,y_8-7,x_9-8,y_9-8,x_10-9,y_10-9,x_11-10,y_11-10,x_12-11,y_12-11,x_13-12,y_13-12,x_14-13,y_14-13,x_15-14,y_15-14,x_16-15,y_16-15,image,d_ratio,d_ratio_sq,sin dif,엄지 중지 내적,검지 중지 내적
0,0.328053,0.052694,1.738621,0.205004,1.710803,0.000000e+00,1.593903,-0.061462,-1.088107,1.711014,1.343636,-0.872529,0.116023,-1.045516,-0.180312,-0.917004,-1.228755,2.474277,1.326146,-0.897170,-0.195602,-1.140285,-0.432859,-1.011401,-0.945779,2.585801,0.526400,-1.002456,-0.954094,-0.798749,-0.776392,-0.630251,correct18.MP4_000003.040.jpg,2.754793,7.588885,-0.410957,-0.357743,0.977981
1,0.432257,0.107098,1.757880,0.286390,1.669664,0.000000e+00,1.500348,-0.032129,-1.224377,1.758906,1.212888,-0.838851,0.128701,-0.991869,-0.217751,-0.904648,-0.988522,2.294526,1.213696,-0.921801,-0.190343,-1.119172,-0.434487,-1.031673,-0.715323,2.541882,0.364314,-1.131044,-1.040938,-0.834963,-0.772113,-0.635486,correct18.MP4_000003.160.jpg,2.474844,6.124852,-0.415779,-0.368311,0.986843
2,1.178796,0.668854,2.240414,0.503644,2.264708,0.000000e+00,2.137628,0.189328,-1.757037,2.066180,1.544756,-1.038888,0.145784,-1.270209,-0.309512,-1.074334,-1.320251,2.588957,1.550659,-1.067472,-0.246939,-1.530433,-0.454991,-1.083492,-1.064814,2.850425,0.795489,-1.301826,-0.964463,-1.362262,-0.656054,-0.754714,correct18.MP4_000003.640.jpg,1.658572,2.750861,-0.180847,-0.467011,0.993155
3,1.144260,0.529248,2.089759,0.401983,1.906068,-1.110223e-16,1.701371,0.002114,-1.536620,1.463530,1.438829,-0.800537,0.197479,-1.080227,-0.242660,-1.015050,-1.391463,2.202594,1.295719,-0.928932,-0.157643,-1.254440,-0.487246,-1.020591,-0.849144,2.504206,0.750748,-1.008786,-0.672528,-1.100871,-0.598922,-0.800808,correct18.MP4_000003.761.jpg,1.147183,1.316029,-0.117288,-0.431956,0.977872
4,1.376628,0.497246,2.061278,0.392072,1.950090,1.110223e-16,1.952590,0.203803,-1.348104,1.907900,1.307060,-1.144669,0.081007,-1.303867,-0.145410,-0.975700,-1.388000,2.663604,1.060249,-1.248017,-0.719432,-1.418663,-0.695723,-0.827909,0.056969,2.723829,0.469721,-1.211811,-1.250295,-1.056907,-0.859611,-0.510950,correct18.MP4_000003.880.jpg,0.820459,0.673153,0.131300,-0.719344,0.852046
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2080,3.016551,-0.381759,3.691754,-0.236746,2.924027,0.000000e+00,2.816820,0.110573,-4.065595,-0.449908,3.123679,0.028073,1.762044,0.334322,1.243812,0.410997,-6.793299,-0.491204,2.935725,-0.189198,1.397921,0.240991,1.136911,0.263412,-6.158255,-0.040337,2.461181,-0.140009,1.266243,0.120035,0.988107,0.153768,not_correct7.mp4_000004.574.jpg,1.083443,1.173849,-0.059374,0.982298,0.995820
2081,3.340392,-1.048243,4.084283,-0.311558,3.232563,-2.775558e-17,2.946834,0.210782,-3.203597,0.555224,3.714328,0.339708,1.902874,0.310285,1.283471,0.240372,-7.142081,-0.518100,3.811456,0.310004,1.910574,0.254058,1.113199,0.337804,-7.431396,-0.609275,3.277226,0.250317,1.894720,0.146645,0.988520,0.151092,not_correct7.mp4_000005.110.jpg,2.589555,6.705795,-0.025514,0.975191,0.994013
2082,2.411187,-0.876213,2.654447,-0.250938,2.151044,6.938894e-18,1.815129,0.142288,-2.325130,0.144753,2.096735,-0.829822,0.288720,-0.939998,-0.436713,-0.695495,-2.032743,2.557503,2.514879,-1.036097,0.190411,-1.208494,-0.761911,-0.928693,-2.254920,3.273475,2.006969,-1.037040,0.059681,-1.190875,-0.470092,-0.882617,not_correct7.mp4_000005.377.jpg,1.089991,1.188080,0.035518,-0.692750,0.992026
2083,1.948865,-0.880748,1.860890,-0.203772,1.413390,-3.469447e-18,1.046627,0.356746,-1.165728,-0.114979,1.782881,-0.466157,0.000875,-0.605097,-0.680646,-0.401592,-1.150040,1.804838,1.881870,-0.420016,-0.448624,-0.711661,-0.907075,-0.512789,-0.672580,1.978611,1.721430,-0.335849,-0.512584,-0.650922,-0.859482,-0.511166,not_correct7.mp4_000005.507.jpg,0.899224,0.808603,0.443340,-0.982747,0.999829


In [ ]:
df_vector_final

,x_1-0,y_1-0,x_2-1,y_2-1,x_3-2,y_3-2,x_4-3,y_4-3,x_5-4,y_5-4,x_6-5,y_6-5,x_7-6,y_7-6,x_8-7,y_8-7,x_9-8,y_9-8,x_10-9,y_10-9,x_11-10,y_11-10,x_12-11,y_12-11,x_13-12,y_13-12,x_14-13,y_14-13,x_15-14,y_15-14,x_16-15,y_16-15,image,d_ratio,d_ratio_sq,sin dif,엄지 중지 내적,검지 중지 내적
0,0.328053,0.052694,1.738621,0.205004,1.710803,0.000000e+00,1.593903,-0.061462,-1.088107,1.711014,1.343636,-0.872529,0.116023,-1.045516,-0.180312,-0.917004,-1.228755,2.474277,1.326146,-0.897170,-0.195602,-1.140285,-0.432859,-1.011401,-0.945779,2.585801,0.526400,-1.002456,-0.954094,-0.798749,-0.776392,-0.630251,correct18.MP4_000003.040.jpg,2.754793,7.588885,-0.410957,-0.357743,0.977981
1,0.432257,0.107098,1.757880,0.286390,1.669664,0.000000e+00,1.500348,-0.032129,-1.224377,1.758906,1.212888,-0.838851,0.128701,-0.991869,-0.217751,-0.904648,-0.988522,2.294526,1.213696,-0.921801,-0.190343,-1.119172,-0.434487,-1.031673,-0.715323,2.541882,0.364314,-1.131044,-1.040938,-0.834963,-0.772113,-0.635486,correct18.MP4_000003.160.jpg,2.474844,6.124852,-0.415779,-0.368311,0.986843
2,1.178796,0.668854,2.240414,0.503644,2.264708,0.000000e+00,2.137628,0.189328,-1.757037,2.066180,1.544756,-1.038888,0.145784,-1.270209,-0.309512,-1.074334,-1.320251,2.588957,1.550659,-1.067472,-0.246939,-1.530433,-0.454991,-1.083492,-1.064814,2.850425,0.795489,-1.301826,-0.964463,-1.362262,-0.656054,-0.754714,correct18.MP4_000003.640.jpg,1.658572,2.750861,-0.180847,-0.467011,0.993155
3,1.144260,0.529248,2.089759,0.401983,1.906068,-1.110223e-16,1.701371,0.002114,-1.536620,1.463530,1.438829,-0.800537,0.197479,-1.080227,-0.242660,-1.015050,-1.391463,2.202594,1.295719,-0.928932,-0.157643,-1.254440,-0.487246,-1.020591,-0.849144,2.504206,0.750748,-1.008786,-0.672528,-1.100871,-0.598922,-0.800808,correct18.MP4_000003.761.jpg,1.147183,1.316029,-0.117288,-0.431956,0.977872
4,1.376628,0.497246,2.061278,0.392072,1.950090,1.110223e-16,1.952590,0.203803,-1.348104,1.907900,1.307060,-1.144669,0.081007,-1.303867,-0.145410,-0.975700,-1.388000,2.663604,1.060249,-1.248017,-0.719432,-1.418663,-0.695723,-0.827909,0.056969,2.723829,0.469721,-1.211811,-1.250295,-1.056907,-0.859611,-0.510950,correct18.MP4_000003.880.jpg,0.820459,0.673153,0.131300,-0.719344,0.852046
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2080,3.016551,-0.381759,3.691754,-0.236746,2.924027,0.000000e+00,2.816820,0.110573,-4.065595,-0.449908,3.123679,0.028073,1.762044,0.334322,1.243812,0.410997,-6.793299,-0.491204,2.935725,-0.189198,1.397921,0.240991,1.136911,0.263412,-6.158255,-0.040337,2.461181,-0.140009,1.266243,0.120035,0.988107,0.153768,not_correct7.mp4_000004.574.jpg,1.083443,1.173849,-0.059374,0.982298,0.995820
2081,3.340392,-1.048243,4.084283,-0.311558,3.232563,-2.775558e-17,2.946834,0.210782,-3.203597,0.555224,3.714328,0.339708,1.902874,0.310285,1.283471,0.240372,-7.142081,-0.518100,3.811456,0.310004,1.910574,0.254058,1.113199,0.337804,-7.431396,-0.609275,3.277226,0.250317,1.894720,0.146645,0.988520,0.151092,not_correct7.mp4_000005.110.jpg,2.589555,6.705795,-0.025514,0.975191,0.994013
2082,2.411187,-0.876213,2.654447,-0.250938,2.151044,6.938894e-18,1.815129,0.142288,-2.325130,0.144753,2.096735,-0.829822,0.288720,-0.939998,-0.436713,-0.695495,-2.032743,2.557503,2.514879,-1.036097,0.190411,-1.208494,-0.761911,-0.928693,-2.254920,3.273475,2.006969,-1.037040,0.059681,-1.190875,-0.470092,-0.882617,not_correct7.mp4_000005.377.jpg,1.089991,1.188080,0.035518,-0.692750,0.992026
2083,1.948865,-0.880748,1.860890,-0.203772,1.413390,-3.469447e-18,1.046627,0.356746,-1.165728,-0.114979,1.782881,-0.466157,0.000875,-0.605097,-0.680646,-0.401592,-1.150040,1.804838,1.881870,-0.420016,-0.448624,-0.711661,-0.907075,-0.512789,-0.672580,1.978611,1.721430,-0.335849,-0.512584,-0.650922,-0.859482,-0.511166,not_correct7.mp4_000005.507.jpg,0.899224,0.808603,0.443340,-0.982747,0.999829
